In [ ]:
#using Pkg
#Pkg.add("GR")

In [ ]:
using Distributions, Plots, GeometryBasics, VoronoiCells
default(size=(800,600))

In [24]:
lambda_c=50.0
lambda_a=5.0

area_a = 4;

Na = 20 #rand(Poisson(lambda_a*area_a))

xa = rand(Uniform(-1,1),Na,2)

Nc = 200 #rand(Poisson(lambda_c*area_a))
xc = rand(Uniform(-1,1),Nc,2);

In [25]:
d(x,y) = sqrt(sum((x-y).^2))
W = [d(xa[i,:],xc[j,:]) for i=1:Na,j=1:Nc]

attachs = zeros(Nc)

for j=1:Nc    
    _,idx = findmin(W[:,j])
    attachs[j] = idx
end
attachs=Int64.(attachs);

In [26]:
rect = Rectangle(Point2(-1.0, -1.0), Point2(1.0, 1.0))
Xa = Point2{Float64}[]
for i=1:size(xa,1)
    aux = Point2(xa[i,1],xa[i,2])
    push!(Xa,aux)
end
tess = voronoicells(Xa, rect);


scatter(xa[:,1],xa[:,2], xlims=(-1,1), ylims=(-1,1), aspectratio=:equal, marker=:square)
scatter!(xc[:,1], xc[:,2], legend=:none, colorbar=:none)

for i=1:Nc
    plot!([xc[i,1],xa[attachs[i],1]], [xc[i,2],xa[attachs[i],2]], color=:black, alpha=0.5)
end

plot!(tess, alpha=0.2, color=:green)

LoadError: UndefVarError: Point2 not defined

In [ ]:
using JuMP, Gurobi

n=size(xa,1)
m=size(xc,1)

model = JuMP.Model(Gurobi.Optimizer)

@variable(model,pi[1:m,1:n]>=0)

@constraint(model, sum(pi,dims=2).== ones(m))
@constraint(model, sum(pi,dims=1).<= 12*ones(1,n))

@objective(model,Min,sum(pi.*W'))

optimize!(model)

sol = value.(pi)

for i=1:Nc    
    _,idx = findmax(sol[i,:])
    attachs[i] = idx
end

attachs = Int64.(attachs);

In [ ]:
scatter(xa[:,1],xa[:,2], xlims=(-1,1), ylims=(-1,1), aspectratio=:equal, marker=:square)
scatter!(xc[:,1], xc[:,2], legend=:none, colorbar=:none)

for i=1:Nc
    plot!([xc[i,1],xa[attachs[i],1]], [xc[i,2],xa[attachs[i],2]], color=:black, alpha=0.5)
end

tess = voronoicells(Xa, rect);
plot!(tess, alpha=0.2, color=:green)